In [8]:
import boto3
aws_lambda = boto3.client('lambda')
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')

In [5]:
import boto3

s3_client = boto3.client('s3')

bucket_name = 'macs123-deployment'

# Check if the bucket exists
bucket_exists = False
try:
    s3_client.head_bucket(Bucket=bucket_name)
    bucket_exists = True
except:
    pass

# Create the bucket if it doesn't exist
if not bucket_exists:
    s3_client.create_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' created successfully.")
else:
    print(f"Bucket '{bucket_name}' already exists.")

Bucket 'macs123-deployment' created successfully.


In [21]:
zip_file_key = 'my_deployment_package.zip'
# Upload the file to S3
with open('deployment.zip', 'rb') as f:
    s3_client.upload_fileobj(f, bucket_name, zip_file_key)

In [22]:
lambda_client = boto3.client('lambda')
try:
    # If function hasn't yet been created, create it
    response = lambda_client.create_function(
        FunctionName='process_chunk',
        Runtime='python3.11',
        Role=role['Role']['Arn'],
        Handler='lambda_function.lambda_handler',
        Code={
            'S3Bucket': bucket_name,
            'S3Key': zip_file_key
        },
        Timeout=300
    )
except lambda_client.exceptions.ResourceConflictException:
    # If function already exists, update it based on the S3 object
    response = lambda_client.update_function_code(
        FunctionName='process_chunk',
        S3Bucket=bucket_name,
        S3Key=zip_file_key
    )
lambda_arn = response['FunctionArn']

In [23]:
import boto3
import json

# Initialize the Lambda client
lambda_client = boto3.client('lambda')

# Define the payload
payload = {
  "chunks": ["test1" , "test2"]
}
 

# Invoke the Lambda function
response = lambda_client.invoke(
    FunctionName='process_chunk',
    InvocationType='RequestResponse',
    Payload=json.dumps(payload)
)

# Print the response
response_payload = json.loads(response['Payload'].read())
print(json.dumps(response_payload, indent=4))

{
    "errorMessage": "Unable to import module 'lambda_function': Error importing numpy: you should not try to import numpy from\n        its source directory; please exit the numpy source tree, and relaunch\n        your python interpreter from there.",
    "errorType": "Runtime.ImportModuleError",
    "requestId": "1a724bfa-112f-41fc-ac62-0f388730e439",
    "stackTrace": []
}


In [34]:
import boto3
from transformers import AutoTokenizer

# Initialize the S3 client
s3_client = boto3.client('s3')
bucket_name = 'macs123-deployment'
# Specify the model identifier
model_identifier = "Minej/bert-base-personality"
# Download the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_identifier)

# Save the tokenizer to a local directory
local_dir = "personality_tokenizer"
tokenizer.save_pretrained(local_dir)

from transformers import BertForSequenceClassification

# Specify the directory where the model files are stored
model_dir = 'personality_model'
# Load the model from the downloaded files
model = BertForSequenceClassification.from_pretrained("Minej/bert-base-personality")
model.save_pretrained(model_dir)


# Process the model outputs as needed


In [ ]:
import os
for file in os.listdir(local_dir):
    s3_client.upload_file(os.path.join(local_dir, file), bucket_name, os.path.join(local_dir, file))

In [35]:
for file in os.listdir(model_dir):
    s3_client.upload_file(os.path.join(model_dir, file), bucket_name, os.path.join(model_dir, file))

In [30]:
s3_resource = boto3.resource('s3')
bucket_name = 'macs123-deployment'
prefix = 'personality_tokenizer/'

# Specify the local directory to save the downloaded files
local_dir = '/tmp/personality_tokenizer/'

# Create the local directory if it doesn't exist
os.makedirs(local_dir, exist_ok=True)
bucket_resource = s3_resource.Bucket(bucket_name)
objects_under_prefix = bucket_resource.objects.filter(Prefix=prefix)

# Iterate over the objects and print their keys
for obj in objects_under_prefix:
    key = obj.key
    filename = os.path.join(local_dir, os.path.basename(key))
    s3_client.download_file(bucket_name, key, filename)
    print(f"Downloaded {key} to {filename}")

Downloaded personality_tokenizer/special_tokens_map.json to /tmp/personality_tokenizer/special_tokens_map.json
Downloaded personality_tokenizer/tokenizer.json to /tmp/personality_tokenizer/tokenizer.json
Downloaded personality_tokenizer/tokenizer_config.json to /tmp/personality_tokenizer/tokenizer_config.json
Downloaded personality_tokenizer/vocab.txt to /tmp/personality_tokenizer/vocab.txt


In [ ]:
s3_resource = boto3.resource('s3')
bucket_name = 'macs123-deployment'
prefix = 'personality_model/'
local_dir = '/tmp/personality_model/'
os.makedirs(local_dir, exist_ok=True)
bucket_resource = s3_resource.Bucket(bucket_name)
objects_under_prefix = bucket_resource.objects.filter(Prefix=prefix)
for obj in objects_under_prefix:
    key = obj.key
    filename = os.path.join(local_dir, os.path.basename(key))
    s3_client.download_file(bucket_name, key, filename)
    print(f"Downloaded {key} to {filename}")



In [31]:
from transformers import BertTokenizer

# Specify the directory where the tokenizer files are stored
tokenizer_dir = '/tmp/personality_tokenizer/'

# Load the tokenizer from the downloaded files
tokenizer = BertTokenizer.from_pretrained(tokenizer_dir)

# Test tokenizer
text = "This is a sample text."
tokens = tokenizer(text)

# Print tokenized output
print(tokens)

{'input_ids': [101, 2023, 2003, 1037, 7099, 3793, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
